# Etude de la détection automatique de caractéristiques sur un signal audio
![ISEN Lille](src/isen.jpg)



Référents :
* A. Frappé 
* A. Gonzalez
* B. Larras

Projet M1 réalisé par :
* J. Le Bellego
* S. Lecoq

# Mise en place

Ce notebook [Jupyter](http://jupyter.org/) se compose de deux types de cellules :

- Celles contenant du **code Python** peuvent être exécuté en appuyant sur simultanément sur les touches *Maj + Entrée*.  
- Celles contenant du **rendu** ou du **markdown** n'ont aucun effet particulier en dehors de contenir du texte.

Le code est prévu pour être exécuté au moins une fois dans l'ordre afin de pouvoir définir tous les éléments, après quoi il vous est possible d'éditer et d'exécutez une celulle en particulier.

**Nota Bene** : Il est possible de masquer certaines portions de code pour rendre le document plus lisible, en cliquant sur l'un des deux boutons ci-dessous.

### Initialisation et dépendances 
Etape intermédiaire consistant à importer les bibliothèques.

In [1]:
%%html
<style>
.output_wrapper button.btn.btn-default, .output_wrapper .ui-dialog-titlebar { display: none; }
.ui-resizable { pointer-events:none; }
</style>
<script>
//!masquer
IPython.OutputArea.prototype._should_scroll = lines => { return false }
function show_code() { $('div.input').show() }   
function hide_code() { $("div.input").each((i, input) => /!masquer/i.test($(input).text()) ? $(input).hide() : $(input).show()) }                 
</script>
<form action="javascript:show_code()" style="float:left"><input type="submit" style="width:200px;padding:4px;border-radius:5px;margin:10px" value="Forcer l'affichage du code"></form>
<form action="javascript:hide_code()" style="float:left"><input type="submit" style="width:200px;padding:4px;border-radius:5px;margin:10px" value="Masquer les codes"></form>

In [2]:
#!masquer
%matplotlib inline
%reload_ext autoreload
%autoreload 2
%matplotlib notebook

In [3]:
#!masquer
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import scipy.io.wavfile as sw
import math
from scipy import signal
from scipy.signal import butter, lfilter
from operator import add
from matplotlib.widgets import Slider
np.seterr(divide='ignore');

In [4]:
# Si actif, cela permettra d'afficher les exemples de démonstration du programme
# Pour tester rapidement, il est conseillé de désactiver cette option
demo_enabled = True

# Fichier audio

La source du fichier audio qui sera utilisée par la démo peut être configurée dans la cellule ci-dessous.

In [5]:
# Source
file = "src/test2.wav"

Ci-dessous sont définit automatiquement les paramètres du fichier source :
* **y** : Signal d'entrée
* **fs** : Fréquence d'échantillonage de ce signal
* **N** : Nombre de points échantillonés
* **t** : Points temporels

In [6]:
# Lecture du fichier audio
fs, y = sw.read(file)
# Nombre d'échantillons et échelle temporelle
N = len(y)
t = np.linspace(0, N/fs, N)

### Spectre d'amplitude

Il représente l'amplitude relative des impulsions sonores (en ordonnée) selon le temps (en abscisse).

Il est affiché par la fonction **plot_specamp(y, t)**.

In [7]:
#! masquer
# Affiche le spectre d'amplitude
# > y : Liste d'amplitudes
# > t : Echelle temporelle
# > [ax] : Surface de dessin (laisser vide pour créer une nouvelle figure)
# > [title] : Titre
# > [color] : Couleur
def plot_specamp(y, t, ax=None, title="Piste audio", color="darkblue"):
    if type(ax) == type(None):
        plt.figure(figsize=(12, 4), dpi= 80, facecolor="w", edgecolor="k")
        ax = plt.subplot(111)
    ax.set_title(title)
    ax.set_xlabel("Temps [s]")
    ax.set_ylabel("Amplitude [Ø]")
    ax.set_xlim(0, t[-1])
    ax.set_ylim(-1, +1)
    ax.plot(t, y, color=color)
    plt.show()

In [8]:
if demo_enabled:
    plot_specamp(y/max(abs(y)), t)

<IPython.core.display.Javascript object>

### Spectrogramme

Il représente la densité spectrale de puissance (par nuances de couleurs) par fréquence (en ordonnée) selon le temps (en abscisse). Les couleurs chaudes indiquent une forte énergie tandis que les couleurs froides en montrent l'absence.

Il est affiché par la fonction **plot_specgram(y, t, fs)**.

In [9]:
#!masquer
# Affiche le spectrogramme
# > y : Liste d'amplitudes
# > t : Echelle temporelle
# > fs : Fréquence d'échantillonage
# > [ax] : Surface de dessin (laisser vide pour créer une nouvelle figure)
def plot_specgram(y, t, fs, ax=None):
    if type(ax) == type(None):
        plt.figure(figsize=(12, 4), dpi= 80, facecolor="w", edgecolor="k")
        ax = plt.subplot(111)
    ax.set_title("Spectrogramme")
    ax.set_ylabel("Frequency [Hz]")
    ax.set_xlabel("Time [s]")
    ax.specgram(y, Fs=fs)
    ax.set_ylim(0, 20000)
    ax.set_xlim(0, t[-1])
    plt.show()

In [10]:
if demo_enabled:
    plot_specgram(y, t, fs)

<IPython.core.display.Javascript object>

# Banque de filtres

Ci-dessous se trouvent les paramètres de la banque de filtres :
* **fmin** : Fréquence centrale minimum
* **fmax** : Fréquence centrale maximum
* **nb_filters** : Nombre de filtres
* **n** : Ordre des filtres
* **q** : Facteur qualité

Ainsi que les paramètres de génération des spectrogrammes :
* **time_res** : Résolution temporelle (∈ [0, 1[), influe sur le *noverlap*
* **amp_res** : Résolution d'amplitude (indiquer le nombre de bits pour stocker l'amplitude)

In [11]:
if demo_enabled:
    # Fréquence minimal
    fmin = 300
    # Fréquence maximal
    fmax = 3000
    # Nombre de filtres
    nb_filters = 16

    # Ordre des filtres
    n = 3
    # Facteur de qualité
    q = 3

    # Résolution temporelle des spectrogramme (entre 0 et 1 exclu, sinon le pc va mourir))
    time_res = 0.1
    # Nombre de bits à utiliser pour pour stocker l'amplitude d'energie
    amp_res = 4

### Génération de la banque de filtres
Etape intermédiaire qui consiste à générer les différents filtres de la banque en fonction des paramètres ci-dessus.

La banque est générée à l'aide de la fonction **gen_filters(q, n, fs, nb_filters=12, fmin=20, fmax=20000, fcs=False, debug=False)** qui effectue plusieurs appels à la fonction **bandpass(fc, q, n, fs, debug=False)**.

Il est également possible de générer une liste de filtres avec des fréquences centrales personnalisées de cette façon : **gen_filters(q, n, fs, fcs=[fc1, fc2, ..., fc3], debug=False)**.

In [12]:
#!masquer
# Génère un filtre passe-bande
# > fc : Fréquence centrale
# > q : Facteur de qualité
# > n : Ordre du filtre
# > fs : Fréquence d'échantillonage
# > [debug] : Si actif, affiche les informations sur le filtre généré
# < filter : Filtre de Butterworth avec les caractéristiques indiquées
# < fc : Fréquence centrale
# < fl : Fréquence de coupure (basse)
# < fh : Fréquence de coupure (haute)
def bandpass(fc, q, n, fs, debug=False):
    # Largeur de la bande passante
    df = fc / q
    # Fréquence de Nyquist
    nyq = fs / 2
    # Fréquences de coupures basses et hautes
    fl = (fc - df/2)
    fh = (fc + df/2)
    # Création du filtre
    if debug: print("Fc : {fc: >4}Hz ({fl: >4}Hz - {fh: >4}Hz)".format(fc=int(fc), fl=int(fl), fh=int(fh)))  
    return butter(N=n, Wn=[fl/nyq, fh/nyq], btype="band"), fc, fl, fh

In [13]:
#!masquer
# Génère la banque des filtres
# Vous pouvez soit préciser des fréquences personnalisées en passant en paramètre une liste : 
# gen_filters(nb_filters, q, n, fs, fcs=[200, 250, 3000], debug=True)
# Soit laissé la fonction générer automatiquement une liste de filtres dans une plage entre fmin et fmax
# > q : Facteur de qualité
# > n : Ordre du filtre
# > fs : Fréquence d'échantillonage
# > [nb_filters] : Nombre de filtres
# > [fmin] : Fréquence minimum
# > [fmax] : Fréquence maximum
# > [fcs] : Fréquence personnalisées
# > [debug] : Si actif, affiche les informations sur le filtre généré
# < filters : Liste des filtres générés
# < filters_fq : Listes d'objets contenant "fc", "fl" et "fh" indiquant les fréquences caractéristiques du filtre associé
def gen_filters(q, n, fs, nb_filters=12, fmin=20, fmax=20000, fcs=False, debug=False):
    # Initialisation
    filters = []; filters_fq = []

    if fcs == False:
        fcs = np.geomspace(fmin, fmax, nb_filters)
    # Création des filtres
    for fc in fcs:
        bp, fc, fl, fh = bandpass(fc, q, n, fs, debug)
        filters.append(bp)
        filters_fq.append({"fc":fc, "fl":fl, "fh":fh})
    return filters, filters_fq

In [14]:
if demo_enabled: #fmin, fmax,
    filters, filters_fq = gen_filters(q, n, fs, nb_filters=16, fmin=fmin, fmax=fmax, debug=True)
    #fcs=[200, 300],

Fc :  300Hz ( 250Hz -  350Hz)
Fc :  349Hz ( 291Hz -  408Hz)
Fc :  407Hz ( 339Hz -  475Hz)
Fc :  475Hz ( 396Hz -  554Hz)
Fc :  554Hz ( 461Hz -  646Hz)
Fc :  646Hz ( 538Hz -  754Hz)
Fc :  753Hz ( 627Hz -  879Hz)
Fc :  878Hz ( 732Hz - 1025Hz)
Fc : 1024Hz ( 853Hz - 1195Hz)
Fc : 1194Hz ( 995Hz - 1393Hz)
Fc : 1392Hz (1160Hz - 1624Hz)
Fc : 1623Hz (1352Hz - 1894Hz)
Fc : 1892Hz (1577Hz - 2208Hz)
Fc : 2206Hz (1839Hz - 2574Hz)
Fc : 2573Hz (2144Hz - 3001Hz)
Fc : 3000Hz (2500Hz - 3500Hz)


### Réponse fréquentielle de la banque de filtres

Le graphique logarithmique ci-dessous affiche les réponses fréquentielle de chaque filtre de la banque sur la plage des sons audibles par l'oreille humaine (20Hz à 20kHz).

Il est affiché par la fonction **plot_freqz(filters)**.

In [15]:
#!masquer
# Affiche la réponse fréquentielle d'une banque de filtre
# > filters : Banque de filtres
def plot_freqz(filters):
    # Figure
    plt.figure(figsize=(12, 6), dpi= 80, facecolor="w", edgecolor="k")
    for i in range(len(filters)):
        # Réponse fréquentielle
        b, a = filters[i]
        w, h = signal.freqz(b, a)
        # Affichage
        plt.semilogx((fs/(2*np.pi))*w, 20 * np.log10(abs(h)));

    # Affichage
    plt.title("Réponse fréquentielle de la banque de filtre")
    plt.xlabel("Fréquence [Hz]")
    plt.ylabel("Gain [dB]")
    plt.xlim([0, 20000])
    plt.ylim([-100, 0])
    plt.grid(which="both", axis="both")
    plt.show()

In [16]:
if demo_enabled:
    plot_freqz(filters)

<IPython.core.display.Javascript object>

### Application de la banque de filtres

Une étape intermédiaire qui consiste à appliquer les différents filtres sur le signal d'entrée.

Les signaux après filtrages sont générés par la fonction **gen_filtered(y, fs, filters)** et l'affichage des signaux filtrés est réalisé par **plot_filtered(y, t, filtered, filters_fq, nsub=4)**.

Le bleu marine représente le signal filtré et le bleu aquatique le signal d'origine.

In [17]:
#!masquer
# Génère les signaux après filtrage
# > y : Liste d'amplitudes
# > fs : Fréquence d'échantillonage
# > filters : Liste de filtres
# < filtered : Liste des signaux filtrés
def gen_filtered(y, fs, filters):
    # Initialisation
    filtered = []; N = len(y);
    t = np.linspace(0, N/fs, N)
    # Application des filtres
    for i in range(len(filters)):
        b, a = filters[i]
        filtered.append(lfilter(b, a, y))
    return filtered

In [18]:
#!masquer
# Affiche les signaux après filtrage
# > y : Signal original
# > t : Echelle temporelle
# > filtered : Liste des signaux filtrés
# > filters_fq : Listes d'objets contenant "fc", "fl" et "fh" indiquant les fréquences caractéristiques du filtre associé
# > [nsub] : Nombre de figures par ligne
def plot_filtered(y, t, filtered, filters_fq, nsub=4):
    # Initialisation
    nl = True; j = 0
    # Affichage
    for i in range(len(filtered)):
        # Figure
        if nl: 
            f, ax = plt.subplots(1, nsub, figsize=(12, 4), dpi= 80, facecolor="w", edgecolor="k")
            nl = False
        j = j +1
        # Affichage du spectre d'amplitude
        ax[i%nsub].set_title("Fc : {fc: >4}Hz".format(fc=int(filters_fq[i]["fc"]), fl=int(filters_fq[i]["fl"]), fh=int(filters_fq[i]["fh"])))
        ax[i%nsub].set_xlabel("Time [s]")
        ax[i%nsub].set_ylabel("Amplitude [Ø]")
        ax[i%nsub].set_xlim(0, t[-1])
        ax[i%nsub].set_ylim(-1, +1)
        ax[i%nsub].plot(t, y/max(abs(y)), color="aqua")
        ax[i%nsub].plot(t, filtered[i]/max(abs(y)), color="darkblue")
        # Nouvelle figure si remplie
        if j%nsub == 0:
            nl = True
            plt.show()
    if j%nsub != 0: plt.show()

In [19]:
if demo_enabled:
    filtered = gen_filtered(y, fs, filters)
    plot_filtered(y, t, filtered, filters_fq)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Etude de l'énergie d'un signal en sortie de la banque
Cette partie permet de définir les fonctions qui permettent de calculer l'énergie d'un signal en sortie de la banque.

La fonction **energies(signal, fs, dt, bits=False)** permet de calculer l'énergie contenue dans un signal segmenté par une certaine résolution temporelle **dt**. Elle effectue de multiples appels à la fonction **energy(signal, fs, start=0, end=False)**.

Il est possible d'afficher ces résulats en appelant directement la fonction **plot_energies(signal, fs, dt, bits)**.

Attention à ne pas utiliser un pas **dt** qui ne pourrait pas être satisfait par la fréquence d'échantillonnage **fs** !

In [20]:
#!masquer
# Calcule l'énergie d'un segment de signal
# > signal : Signal
# > fs : Fréquence d'échantillonage
# > [start] : Départ (en sec) 
# > [end] : Fin (en sec)
# < seq : Energie contenue dans la séquence de t=start à t=end
def energy(signal, fs, start=0, end=False):
    # Indices
    start = int(start * fs)
    if end == False:
        end = len(signal)
    else:
        end = int(min(end * fs, len(signal)-1))
    # Energie
    seq = signal[start:end]
    return sum(abs(seq) ** 2 )

In [21]:
#!masquer
# Calcule l'énergie contenue dans un signal segmenté par une certaine résolution temporelle
# > signal : Signal
# > fs : Fréquence d'échantillonage
# > dt : Résolution temporelle (Celle-ci doit évidemment pouvoir être satisfaire par la valeur de fs)
# > [bits] : Nombre de bits sur lequel est codé la valeur
# < segs : Segments temporelles
# < seqs : Energie contenue dans chaque segment temporel
def energies(signal, fs, dt, bits=False):
    # Energie
    seqs = []
    segs = np.arange(0, len(signal)/fs, dt)
    for t in segs:
        seqs.append(energy(signal, fs, t, t+dt))
    # Codage
    if bits != False:
        seqs = np.round(np.array(seqs)/(np.max(seqs)/(2**bits-1)))
    return segs, seqs

In [22]:
#!masquer
# Affiche l'énergie contenue dans un signal segmenté par une certaine résolution temporelle
# > signal : Signal
# > fs : Fréquence d'échantillonage
# > dt : Résolution temporelle (Celle-ci doit évidemment pouvoir être satisfaire par la valeur de fs)
# > [bits] : Nombre de bits sur lequel est codé la valeur
def plot_energies(signal, fs, dt, bits):
    plt.figure(figsize=(12, 2), dpi= 80, facecolor="w", edgecolor="k")
    # Energie (les valeurs sont dupliquées juste pour l'affichage)
    segs, seqs = energies(signal, fs, dt, bits)
    X, Y = np.meshgrid(segs, [0, 1])
    plt.pcolormesh(X, Y, [seqs, seqs], cmap="magma")
    # Affichage
    plt.colorbar(orientation="horizontal")
    plt.tick_params(axis="y", which="both", left="off", labelleft="off")
    plt.title("Energie du signal par segment de {dt}s sur {bits}bits".format(dt=dt, bits=bits))
    plt.show()

In [23]:
if demo_enabled:
    plot_energies(filtered[1], fs, 0.01, bits=2)

<IPython.core.display.Javascript object>


# Etude de l'énergie des signaux en sortie de la banque
Cette partie affiche le signal audio en sortie, le spectrogramme associé ainsi que les états après chaque application respective des filtres de la banque.

Les données du spectrogramme sont calculés par la fonction **gen_data(filtered, fs, time_res, amp_res, filters_fq)** puis affichées par la fonction **plot_data(y, t, rsegs, rfreqs, rseqs, ax=None)**.

In [24]:
#!masquer
# Calcul du spectrogramme
# > filtered : Liste de signaux filtrés
# > fs : Fréquence d'échantillonage
# > time_res : Résolution temporelle
# > amp_res : Résolution en amplitude
# > filters_fq : Listes d'objets contenant "fc", "fl" et "fh" indiquant les fréquences caractéristiques du filtre associé
# < rsegs : Liste des segments temporels 
# < rfreqs : Liste de fréquences
# < rseqs : Liste des séquence d'énergie
def gen_data(filtered, fs, time_res, amp_res, filters_fq):    
    # Initialisation
    rsegs = [] ; rseqs = [] ; rfreqs = [] 
    # Spectrogramme
    for i in range(len(filtered)):
        rsegs, seqs = energies(filtered[i], fs=fs, dt=time_res)
        rseqs.append(seqs)
        rfreqs.append(filters_fq[i]["fc"])
    # Normalisation
    for i in range(len(rseqs)):
        for j in range(len(rseqs[i])):
            rseqs[i] = np.round(np.array(rseqs[i])/(np.max(rseqs[i])/(2**amp_res-1)))
    return rsegs, rfreqs, rseqs

In [25]:
#! masquer
# Affiche le spectrogramme personnalisable
# > rsegs : Liste des segments temporels 
# > rfreqs : Liste de fréquences
# > rseqs : Liste des séquence d'énergie
# > [ax] : Figures à réutiliser
def plot_datagram(rsegs, rfreqs, rseqs, ax=None):
    if type(ax) == type(None):
        plt.figure(figsize=(12, 4), dpi= 80, facecolor="w", edgecolor="k")
        ax = plt.subplot(111)
    X, Y = np.meshgrid(rsegs, rfreqs)
    ax.set_title("Spectrogramme")
    ax.set_xlabel("Time [s]")
    ax.set_ylabel("Frequency [Hz]")
    ax.set_yscale("log")
    ax.set_yticks(np.geomspace(rfreqs[0], rfreqs[-1], len(rfreqs)))
    ax.get_yaxis().set_major_formatter(matplotlib.ticker.FormatStrFormatter('%d'))
    ax.get_yaxis().set_minor_formatter(matplotlib.ticker.NullFormatter())
    ax.pcolormesh(X, Y, rseqs, cmap="magma")
    ax.set_xlim(0, rsegs[-1])
    ax.plot()
    plt.show()

In [26]:
#!masquer
# Affiche les données audio du signal
# > y : Signal d'entrée
# > t : Echelle temporelle
# > rsegs : Liste des segments temporels 
# > rfreqs : Liste de fréquences
# > rseqs : Liste des séquence d'énergie
# > [ax] : Figures à réutiliser
def plot_data(y, t, rsegs, rfreqs, rseqs, ax=None):
    # Figure
    if type(ax) == type(None):
        f, ax = plt.subplots(2, 1, figsize=(12, 8), dpi= 80, facecolor="w", edgecolor="k")
        
    # Spectre d'amplitude
    plot_specamp(y/max(abs(y)), t, ax=ax[0])

    # Spectrogramme
    plot_datagram(rsegs, rfreqs, rseqs, ax=ax[1])

In [27]:
if demo_enabled:
    rsegs, rfreqs, rseqs = gen_data(filtered, fs, time_res, amp_res, filters_fq)
    plot_data(y, t, rsegs, rfreqs, rseqs)

<IPython.core.display.Javascript object>

TypeError: unsupported operand type(s) for /: 'int' and 'list'

### Etude des états

Afin de faciliter le traitement des données, il est possible de lire la valeur d'un retournée par l'état d'un filtre à un instant *t* donnée en utilisant la fonction **state_at(filter_no, s, rsegs, rseqs, debug=False)**.

In [ ]:
#!masquer
# Récupère la valeur retournée par un filtre à un temps donnée
# > filter_no : Indice du filtre
# > s : Temps (valeur comprise entre 0 et la durée du signal audio)
# > rsegs : Liste des segments temporels 
# > rseqs : Liste des séquence d'énergie
# > [debug] : Si actif, affiche les informations sur l'état récupéré
# < state : Valeur numérique du filtre pour t=s
def state_at(filter_no, s, rsegs, rseqs, debug=False):
    # Calcul de l'indice dans la séquence
    i = 0
    while (i < len(rsegs)) and (rsegs[i] < s):
        i = i+1
    # Retourne l'état du filtre au temps spécifié
    if debug: print("Valeur numérique du filtre n°{filter_no} pour t={t}s".format(filter_no=filter_no, t=rsegs[i])) 
    return rseqs[filter_no][i]

In [ ]:
if demo_enabled:
    print(state_at(12, 0.18, rsegs, rseqs, debug=True))

# Récapitulatif

Il est possible d'exécuter toutes les fonctions ci-dessus avec la fonction **compute(file, fs=0, time_res=0, amp_res=0, filters=False, fmin=0, fmax=0, nb_filters=0, q=0, n=0)**.

Afin de réduire les temps de calculs, il est possible d'exécuter cette fonction de différentes façons.

### **Sélection de la source**
* **file** : Source du fichier à ouvrir (string)

OU 

* **file** : Signal d'entrée (liste d'amplitudes)
* **fs** : Fréquence d'échantillonage

### **Sélection de la banque de filtres**
* **filters** : Banque de filtre déjà généré (permet d'éviter de les regénérer à chaque fois)
* **filters_fq** : Données caractéristiques des filtres déjà générés

OU

* **fmin** : Fréquence minimum
* **fmax** : Fréquence maximum
* **nb_filters** : Nombre de filtres
* **q** : Facteur de qualité
* **n** : Ordre du filtre

OU

* **fcs** : Liste de fréquences centrales personnalisées
* **q** : Facteur de qualité
* **n** : Ordre du filtre

### **Configuration du spectrogramme**
* **time_res** : Résolution temporelle
* **amp_res** : Résolution en amplitude

### **Utilisation d'une figure déjà existante**
* **f** : Figure contenant ax (laisser vide si le paramètre ax n'est pas spécifié)
* **ax** : Surface de dessin existante (laisser vide pour créer une nouvelle figure)


In [ ]:
#!masquer
# Exécute le code dans sa totalité.
# Cette fonction peut peut être utilisé de plusieurs façons avant d'optimiser les temps de calculs.
#
# Sélection de la source
#    > file : Source du fichier à ouvrir (string)
#    OU 
#    > file : Signal d'entrée (liste d'amplitudes)
#    > fs : Fréquence d'échantillonage
#
# Sélection de la banque de filtres
#    > filters : Banque de filtre déjà généré (permet d'éviter de les regénérer à chaque fois)
#    > filters_fq : Données caractéristiques des filtres déjà générés
#    OU 
#    > fmin : Fréquence minimum
#    > fmax : Fréquence maximum
#    > nb_filters : Nombre de filtres
#    > q : Facteur de qualité
#    > n : Ordre du filtre
#    OU
#    > fcs : Liste de fréquences centrales personnalisées
#    > q : Facteur de qualité
#    > n : Ordre du filtre
#
# Configuration du spectrogramme
#    > time_res : Résolution temporelle
#    > amp_res : Résolution en amplitude
#
# Utilisation d'une figure déjà existante
#    > [ax] : Surface de dessin existante (laisser vide pour créer une nouvelle figure)
#
# < ax : Figure secondaire généré par la fonction gen_data
# < y : Signal d'entrée
# < t : Echelle temporelle
# < rspectrum : Spectre généré par la fonction gen_data
# < rfreqs : Liste de fréquences généré par la fonction gen_data
# < rtime : Liste de points temporels généré par la fonction gen_data
def compute(file, fs=0, time_res=0, amp_res=0, fmin=0, fmax=0, fcs=False, nb_filters=0, q=0, n=0, filters=[], filters_fq=[], ax=None):
    # Récupération du fichier audio
    if type(file) == str:
        fs, y = sw.read(file)
    else:
        y = file
    N = len(y)
    t = np.linspace(0, N/fs, N)
    
    # Filtrage
    if nb_filters > 0:
        filters, filters_fq = gen_filters(q, n, fs, nb_filters=nb_filters, fmin=fmin, fmax=fmax, fcs=fcs)
    filtered = gen_filtered(y, fs, filters)
    
    # Spectrogramme
    rsegs, rfreqs, rseqs = gen_data(filtered, fs, time_res, amp_res, filters_fq)
    plot_data(y, t, rsegs, rfreqs, rseqs, ax=ax)
    return rsegs, rfreqs, rseqs

Par exemple, pour charger un fichier, générer une banque de filtre et afficher le spectre d'amplitude ainsi que le spectrogramme, il suffit d'écrire :

In [ ]:
if demo_enabled:
    compute(
        file="src/Words/voice_F1_absolutely_2.wav", 
        fmin=300, 
        fmax=3000, 
        nb_filters=16, 
        q=3, 
        n=3, 
        time_res=0.05, 
        amp_res=2
    )

Le même exemple avec les mêmes paramètres hormis qu'il s'agit d'une autre personne prononçant le même mot :

In [ ]:
if demo_enabled:
    compute(
        file="src/Words/voice_F2_absolutely_2.wav", 
        fmin=300, 
        fmax=3000, 
        nb_filters=16, 
        q=3, 
        n=3, 
        time_res=0.05, 
        amp_res=2
    )

# Traitement à la volée

Si **[pyaudio](https://people.csail.mit.edu/hubert/pyaudio/)** est installé sur votre machine, il est possible d'enregistrer des sons depuis votre microphone et de les traiter au fur et à mesure (avec un léger décalage).

Pour cela, assurer vous de configurer correctement les deux variables suivantes, ce qui permettra d'inclure de nouvelles bibliothèques requises pour faire fonctionner la suite de ce programme.

Vous pouvez également configurer plus en détail les différents paramètres pour l'enregistrement, même s'il est conseillé de les laisser tel quel.

In [ ]:
# Chemin vers les bibliothèques Python
lib_path = "C:\Program Files\Python36\Lib\site-packages"

# Chemin où stocker le fichier audio généré
src_out = "src/output.wav"

In [ ]:
#!masquer
import sys
sys.path.append(lib_path)
import pyaudio
import wave
import time
from IPython.display import clear_output

In [ ]:
#!masquer
# Format d'enregistrement
dformat = pyaudio.paInt16
# Nombre de canaux (mono est conseillé)
channels = 1
# Fréquence d'échantillonage
fs = 48000
# Taille des blocs enregistrés
chunk_size = 1024

In [ ]:
# Durée de l'enregistrement
duration = 0.25

L'enregistrement est réalisé par la fonction **live_record()**.

In [ ]:
#!masquer
# Commence un enregistrement vocal live
# Afin d'éviter les calculs liés à la création des filtres, il est nécessaire de générer les filtres au préalable
# > filters : Banque de filtre déjà généré (permet d'éviter de les regénérer à chaque fois)
# > filters_fq : Données caractéristiques des filtres déjà générés
# > time_res : Résolution temporelle
# > amp_res : Résolution en amplitude
# 
# Note : Il est peut-être possible de traiter le flux à la volée
#
def live_record(filters, filters_fq, time_res, amp_res):
    # Initialisation
    plt.ion() 
    f, ax = plt.subplots(2, 1, figsize=(24, 12), dpi= 80, facecolor="w", edgecolor="k")
    f.show()
    
    # Frames
    frames = []
    
    # Boucle principale
    while True:
        # Ouverture du flux
        p = pyaudio.PyAudio()
        stream = p.open(format=dformat, channels=channels, rate=fs, input=True, frames_per_buffer=chunk_size)
    
        # Enregistrement
        for i in range(0, int(fs / chunk_size * duration)):
            data = stream.read(chunk_size)
            frames.append(data)
        frames = frames[-int(fs / chunk_size * 5):]

        # Fermeture du flux
        stream.stop_stream()
        stream.close()
        p.terminate()

        # Ecriture du fichier
        wf = wave.open(src_out, "wb")
        wf.setnchannels(channels)
        wf.setsampwidth(p.get_sample_size(dformat))
        wf.setframerate(fs)
        wf.writeframes(b''.join(frames))
        wf.close()
    
        # Lecture et traitement
        nfs, y = sw.read(src_out)
        ax[0].clear()
        ax[1].clear()
        compute(file=src_out, filters=filters, filters_fq=filters_fq, time_res=time_res, amp_res=amp_res, ax=ax)
        # Affichage
        f.canvas.draw()
    return

Le code suivant permet de commencer un nouvel enregistrement et de traiter en temps réel les données.

Il faut **interrompre** le kernel afin de pouvoir arrêter l'exécution de cette cellule et pouvoir en utiliser d'autres :
![Interruption](src/stop.png)

In [ ]:
try:
    # Paramètrage de l'enregistrement live
    filters, filters_fq = gen_filters(fmin=300, fmax=3000, nb_filters=16, q=3, n=3, fs=fs)
    live_record(time_res=0.1, amp_res=2, filters_fq=filters_fq, filters=filters)
    
# Masque l'erreur en cas d'interruption du Kernel
except KeyboardInterrupt:
    clear_output() ; print("Terminé !") ; quit

# TODO
* Vérifier si l'exploitation des amplitudes suffit ?
* Ajouter un loader qui permet de comparer différents fichiers sans action manuelles